# -----------------------

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau
import sklearn.metrics  


In [2]:
import pandas as pd

In [3]:
dev=pd.read_csv('/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/WavLM-EMOFAKE/Chinese_dev_WavLM.csv')
test=pd.read_csv('/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/WavLM-EMOFAKE/Chinese_test_WavLM.csv')
train=pd.read_csv('/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/WavLM-EMOFAKE/Chinese_train_WavLM.csv')

In [4]:
dev.columns

Index(['filename', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       ...
       '759', '760', '761', '762', '763', '764', '765', '766', '767', 'class'],
      dtype='object', length=770)

In [5]:
test.columns

Index(['filename', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       ...
       '759', '760', '761', '762', '763', '764', '765', '766', '767', 'class'],
      dtype='object', length=770)

In [6]:
train.columns

Index(['filename', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       ...
       '759', '760', '761', '762', '763', '764', '765', '766', '767', 'class'],
      dtype='object', length=770)

In [8]:
dev.head()

,filename,0,1,2,3,4,5,6,7,8,...,759,760,761,762,763,764,765,766,767,class
0,S2_Surprise2Angry_0010_001420.wav,0.189128,-0.209342,-0.004541,0.491101,-0.249494,-0.100917,0.049525,0.006354,-0.017165,...,-0.222937,-0.228415,0.339616,-0.211259,-0.034513,0.037573,0.223898,0.040566,-0.147796,spoof
1,0009_Neutral_000264.wav,0.172095,-0.198275,0.066159,0.427224,-0.211104,-0.072713,0.121893,-0.114145,-0.076951,...,-0.226404,-0.208485,0.293772,-0.133162,-0.083665,0.085294,0.206235,-0.032916,-0.226173,bonafide
2,0010_Angry_000683.wav,0.184614,-0.222184,0.001011,0.484833,-0.242463,-0.066548,0.090960,-0.079036,-0.060983,...,-0.248600,-0.226518,0.309246,-0.163002,-0.067097,0.061516,0.215943,-0.000802,-0.142232,bonafide
3,0009_Surprise_001651.wav,0.193536,-0.199418,0.019337,0.477495,-0.226425,-0.087098,0.083318,0.000034,-0.027949,...,-0.214360,-0.261837,0.307350,-0.191124,-0.070351,0.052526,0.201908,0.021133,-0.130850,bonafide
4,S2_Surprise2Neutral_0009_001456.wav,0.184930,-0.204854,-0.033367,0.522085,-0.273352,-0.075318,0.064707,0.013866,-0.031558,...,-0.159633,-0.238744,0.253641,-0.203063,-0.029254,0.033224,0.180608,0.033090,-0.120810,spoof


In [9]:
test.head()

,filename,0,1,2,3,4,5,6,7,8,...,759,760,761,762,763,764,765,766,767,class
0,0008_Happy_000882.wav,0.160846,-0.202603,-0.029431,0.452209,-0.256997,-0.039944,0.024706,0.023729,-0.011878,...,-0.180685,-0.249422,0.183773,-0.200146,-0.057717,0.020661,0.199693,0.009091,-0.093136,bonafide
1,S7_Sad2Surprise_0008_001054.wav,0.158242,-0.217189,-0.025827,0.468232,-0.225677,-0.085270,0.022428,0.055285,0.042900,...,-0.196830,-0.277372,0.293661,-0.198013,-0.036369,0.030010,0.206655,0.038444,-0.128283,spoof
2,S5_Sad2Angry_0008_001358.wav,0.300717,-0.307127,0.067005,0.634999,-0.274562,-0.142308,0.084050,-0.094868,0.012020,...,-0.342937,-0.229861,0.933153,-0.219925,-0.061918,0.098574,0.346627,0.086339,-0.178897,spoof
3,S7_Sad2Angry_0008_001324.wav,0.162972,-0.258586,0.018852,0.473129,-0.274531,-0.085156,0.070843,-0.017749,-0.029502,...,-0.272154,-0.232923,0.415868,-0.192512,-0.059498,0.054649,0.270987,0.019338,-0.149768,spoof
4,S5_Sad2Happy_0008_001242.wav,0.207343,-0.298982,0.052051,0.449070,-0.257353,-0.134474,0.098110,-0.093671,-0.066941,...,-0.328760,-0.170260,0.545209,-0.195658,-0.058123,0.077800,0.319440,0.032278,-0.089926,spoof


In [10]:
train.head()

,filename,0,1,2,3,4,5,6,7,8,...,759,760,761,762,763,764,765,766,767,class
0,S1_Angry2Happy_0004_000578.wav,0.192144,-0.230899,-0.007889,0.463364,-0.309516,-0.119908,0.047797,0.007947,-0.044678,...,-0.248091,-0.187253,0.327632,-0.248353,-0.045072,0.034011,0.238101,0.035052,-0.160326,spoof
1,S2_Happy2Angry_0005_001020.wav,0.164120,-0.207680,-0.018232,0.457945,-0.255097,-0.084890,0.056021,0.022304,-0.050971,...,-0.187700,-0.247315,0.268146,-0.217452,-0.067052,0.028347,0.188930,0.023942,-0.205236,spoof
2,S1_Neutral2Surprise_0001_000215.wav,0.185979,-0.267814,0.030250,0.406555,-0.268612,-0.113836,0.106833,-0.081320,-0.076577,...,-0.286851,-0.213667,0.461657,-0.176494,-0.071047,0.060256,0.274009,0.011276,-0.211352,spoof
3,0006_Happy_000827.wav,0.193432,-0.242063,0.019229,0.464806,-0.215070,-0.085839,0.088731,-0.027413,-0.025742,...,-0.281962,-0.227900,0.332402,-0.189493,-0.079213,0.063780,0.249012,0.013761,-0.127866,bonafide
4,S2_Neutral2Surprise_0001_000313.wav,0.156264,-0.239932,0.005776,0.503117,-0.245035,-0.068548,0.067865,-0.045244,-0.044124,...,-0.218631,-0.249604,0.376530,-0.202699,-0.070528,0.045023,0.216239,-0.004558,-0.150248,spoof


In [ ]:

def load_and_preprocess_data(train_file, dev_file, test_file):
    train = pd.read_csv(train_file)
    dev = pd.read_csv(dev_file)
    test = pd.read_csv(test_file)
    
    X_train = train.iloc[:, 1:-1].values
    y_train = train['class'].values

    X_dev = dev.iloc[:, 1:-1].values
    y_dev = dev['class'].values

    X_test = test.iloc[:, 1:-1].values
    y_test = test['class'].values

    encoder = LabelEncoder()
    y_train = encoder.fit_transform(y_train)
    y_dev = encoder.transform(y_dev)
    y_test = encoder.transform(y_test)

    X_train = np.expand_dims(X_train, axis=2)
    X_dev = np.expand_dims(X_dev, axis=2)
    X_test = np.expand_dims(X_test, axis=2)
    
    return X_train, y_train, X_dev, y_dev, X_test, y_test

def compute_eer(label, pred, positive_label=1):
    fpr, tpr, threshold = sklearn.metrics.roc_curve(label, pred, pos_label=positive_label)
    fnr = 1 - tpr
    eer_threshold = threshold[np.nanargmin(np.absolute((fnr - fpr)))]
    eer_1 = fpr[np.nanargmin(np.absolute((fnr - fpr)))]
    eer_2 = fnr[np.nanargmin(np.absolute((fnr - fpr)))]
    eer = (eer_1 + eer_2) / 2
    return eer

def build_cnn_model(input_shape, initial_learning_rate=0.001):
    model = Sequential([
        Conv1D(64, kernel_size=3, activation='relu', input_shape=input_shape),
        BatchNormalization(), 
        MaxPooling1D(pool_size=2),
        Dropout(0.2),   
        Conv1D(128, kernel_size=3, activation='relu'),
        BatchNormalization(),  
        MaxPooling1D(pool_size=2),
        Dropout(0.2),
        Flatten(),
        Dense(128, activation='relu'),
        BatchNormalization(),  
        Dropout(0.5),
        Dense(2, activation='softmax')  
    ])
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=initial_learning_rate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

def train_and_evaluate_cnn(train_file, dev_file, test_file, epochs=10, batch_size=32, initial_learning_rate=0.001, model_save_path="cnn_MMS.h5"):
    X_train, y_train, X_dev, y_dev, X_test, y_test = load_and_preprocess_data(train_file, dev_file, test_file)
    
    model = build_cnn_model(input_shape=(X_train.shape[1], 1), initial_learning_rate=initial_learning_rate)
  
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
    
    model.fit(
        X_train, y_train, 
        validation_data=(X_dev, y_dev), 
        epochs=epochs, 
        batch_size=batch_size,
        callbacks=[reduce_lr]
    )
    
    y_pred_prob = model.predict(X_test)[:, 1]  
    y_pred = np.argmax(model.predict(X_test), axis=1)
    print(classification_report(y_test, y_pred, target_names=['bonafide', 'spoof'], digits=4))

    eer = compute_eer(y_test, y_pred_prob, positive_label=1)
    eer_percentage = eer * 100  
    print(f'Equal Error Rate (EER): {eer_percentage:.20f}%')
    model.save(model_save_path)
    print(f"Model saved at {model_save_path}")
    
    return model


# C WAVLM

In [ ]:

train_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/WavLM-EMOFAKE/Chinese_train_WavLM.csv'
dev_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/WavLM-EMOFAKE/Chinese_dev_WavLM.csv'
test_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/WavLM-EMOFAKE/Chinese_test_WavLM.csv'

cnn_model = train_and_evaluate_cnn(train_file, dev_file, test_file, epochs=100, batch_size=32)


2024-10-04 22:25:47.803976: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-04 22:25:48.219136: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-04 22:25:48.219218: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-04 22:25:48.287310: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-04 22:25:48.426379: I tensorflow/core/platform/cpu_feature_guar

Epoch 1/100


2024-10-04 22:25:59.456055: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-10-04 22:25:59.776057: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-10-04 22:26:00.293530: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-10-04 22:26:01.385578: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f0245045f10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-10-04 22:26:01.385663: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 6GB Laptop GPU, Compute Capability 8.6
2024-10-04 22:26:01.404829: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1728060961.496860    1438 d

854/854 [==============================] - 20s 18ms/step - loss: 0.3150 - accuracy: 0.8681 - val_loss: 0.3170 - val_accuracy: 0.8627 - lr: 0.0010
Epoch 2/100
854/854 [==============================] - 15s 17ms/step - loss: 0.2358 - accuracy: 0.9018 - val_loss: 0.7733 - val_accuracy: 0.6913 - lr: 0.0010
Epoch 3/100
854/854 [==============================] - 14s 17ms/step - loss: 0.2173 - accuracy: 0.9113 - val_loss: 0.4878 - val_accuracy: 0.7953 - lr: 0.0010
Epoch 4/100
854/854 [==============================] - 15s 18ms/step - loss: 0.2101 - accuracy: 0.9149 - val_loss: 0.2953 - val_accuracy: 0.8786 - lr: 0.0010
Epoch 5/100
854/854 [==============================] - 15s 17ms/step - loss: 0.2118 - accuracy: 0.9151 - val_loss: 0.2271 - val_accuracy: 0.9076 - lr: 0.0010
Epoch 6/100
854/854 [==============================] - 15s 18ms/step - loss: 0.2007 - accuracy: 0.9166 - val_loss: 0.2102 - val_accuracy: 0.9164 - lr: 0.0010
Epoch 7/100
854/854 [==============================] - 15s 17ms/

# E wavlm

In [1]:

train_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/WavLM-EMOFAKE/English_train_WavLM.csv'
dev_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/WavLM-EMOFAKE/English_dev_WavLM.csv'
test_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/WavLM-EMOFAKE/English_test_WavLM.csv'

cnn_model = train_and_evaluate_cnn(train_file, dev_file, test_file, epochs=100, batch_size=32)


2024-10-04 23:06:04.051309: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-04 23:06:04.415704: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-04 23:06:04.415801: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-04 23:06:04.476434: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-04 23:06:04.595666: I tensorflow/core/platform/cpu_feature_guar

Epoch 1/100


2024-10-04 23:06:13.358160: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-10-04 23:06:13.602820: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-10-04 23:06:13.968271: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-10-04 23:06:14.866341: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fc709dbdd80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-10-04 23:06:14.866386: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 6GB Laptop GPU, Compute Capability 8.6
2024-10-04 23:06:14.881670: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1728063374.961321    1258 d

854/854 [==============================] - 17s 15ms/step - loss: 0.3736 - accuracy: 0.8376 - val_loss: 0.3172 - val_accuracy: 0.8441 - lr: 0.0010
Epoch 2/100
854/854 [==============================] - 11s 13ms/step - loss: 0.2764 - accuracy: 0.8832 - val_loss: 0.1234 - val_accuracy: 0.9569 - lr: 0.0010
Epoch 3/100
854/854 [==============================] - 11s 13ms/step - loss: 0.2550 - accuracy: 0.8944 - val_loss: 0.1068 - val_accuracy: 0.9670 - lr: 0.0010
Epoch 4/100
854/854 [==============================] - 11s 13ms/step - loss: 0.2373 - accuracy: 0.9040 - val_loss: 0.1063 - val_accuracy: 0.9608 - lr: 0.0010
Epoch 5/100
854/854 [==============================] - 12s 14ms/step - loss: 0.2239 - accuracy: 0.9084 - val_loss: 0.0698 - val_accuracy: 0.9789 - lr: 0.0010
Epoch 6/100
854/854 [==============================] - 15s 17ms/step - loss: 0.2155 - accuracy: 0.9143 - val_loss: 0.0789 - val_accuracy: 0.9788 - lr: 0.0010
Epoch 7/100
854/854 [==============================] - 14s 17ms/

# C XVECTOR

In [5]:

train_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/xvector-EMOFAKE/Chinese_train_xvector.csv'
dev_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/xvector-EMOFAKE/Chinese_dev_xvector.csv'
test_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/xvector-EMOFAKE/Chinese_test_xvector.csv'

cnn_model = train_and_evaluate_cnn(train_file, dev_file, test_file, epochs=100, batch_size=32)


Epoch 1/100
854/854 [==============================] - 16s 17ms/step - loss: 0.0207 - accuracy: 0.9932 - val_loss: 0.0019 - val_accuracy: 0.9996 - lr: 0.0010
Epoch 2/100
854/854 [==============================] - 15s 17ms/step - loss: 0.0064 - accuracy: 0.9981 - val_loss: 0.0100 - val_accuracy: 0.9963 - lr: 0.0010
Epoch 3/100
854/854 [==============================] - 14s 16ms/step - loss: 0.0061 - accuracy: 0.9981 - val_loss: 0.0082 - val_accuracy: 0.9970 - lr: 0.0010
Epoch 4/100
854/854 [==============================] - 14s 16ms/step - loss: 0.0065 - accuracy: 0.9982 - val_loss: 0.0750 - val_accuracy: 0.9732 - lr: 0.0010
Epoch 5/100
854/854 [==============================] - 14s 17ms/step - loss: 0.0051 - accuracy: 0.9984 - val_loss: 0.0051 - val_accuracy: 0.9981 - lr: 5.0000e-04
Epoch 6/100
854/854 [==============================] - 15s 17ms/step - loss: 0.0026 - accuracy: 0.9993 - val_loss: 1.9391e-04 - val_accuracy: 0.9999 - lr: 5.0000e-04
Epoch 7/100
854/854 [===================

# E Xvector

In [6]:

train_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/xvector-EMOFAKE/English_train_xvector.csv'
dev_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/xvector-EMOFAKE/English_dev_xvector.csv'
test_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/xvector-EMOFAKE/English_test_xvector.csv'

# Call the function to train and evaluate the CNN model
cnn_model = train_and_evaluate_cnn(train_file, dev_file, test_file, epochs=100, batch_size=32)


Epoch 1/100
854/854 [==============================] - 14s 14ms/step - loss: 0.0201 - accuracy: 0.9930 - val_loss: 0.0230 - val_accuracy: 0.9930 - lr: 0.0010
Epoch 2/100
854/854 [==============================] - 11s 13ms/step - loss: 0.0096 - accuracy: 0.9972 - val_loss: 0.0190 - val_accuracy: 0.9940 - lr: 0.0010
Epoch 3/100
854/854 [==============================] - 11s 13ms/step - loss: 0.0077 - accuracy: 0.9978 - val_loss: 0.0050 - val_accuracy: 0.9982 - lr: 0.0010
Epoch 4/100
854/854 [==============================] - 11s 13ms/step - loss: 0.0097 - accuracy: 0.9969 - val_loss: 0.0328 - val_accuracy: 0.9875 - lr: 0.0010
Epoch 5/100
854/854 [==============================] - 11s 13ms/step - loss: 0.0054 - accuracy: 0.9985 - val_loss: 1.6564 - val_accuracy: 0.6999 - lr: 0.0010
Epoch 6/100
854/854 [==============================] - 11s 13ms/step - loss: 0.0075 - accuracy: 0.9975 - val_loss: 0.0017 - val_accuracy: 0.9995 - lr: 0.0010
Epoch 7/100
854/854 [==============================]

# E MERT-v1-330M

In [8]:
train_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/MERT-330/English_train_MERT-v1-330M/English_train_MERT-v1-330M.csv'
dev_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/MERT-330/English_dev_MERT-v1-330M/English_dev_MERT-v1-330M.csv'
test_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/MERT-330/English_test_MERT-v1-330M/English_test_MERT-v1-330M.csv'

# Call the function to train and evaluate the CNN model
cnn_model = train_and_evaluate_cnn(train_file, dev_file, test_file, epochs=100, batch_size=32)


Epoch 1/100
854/854 [==============================] - 20s 20ms/step - loss: 0.2575 - accuracy: 0.8905 - val_loss: 0.1174 - val_accuracy: 0.9541 - lr: 0.0010
Epoch 2/100
854/854 [==============================] - 15s 18ms/step - loss: 0.1007 - accuracy: 0.9609 - val_loss: 0.0872 - val_accuracy: 0.9652 - lr: 0.0010
Epoch 3/100
854/854 [==============================] - 15s 18ms/step - loss: 0.0811 - accuracy: 0.9706 - val_loss: 0.0735 - val_accuracy: 0.9715 - lr: 0.0010
Epoch 4/100
854/854 [==============================] - 15s 18ms/step - loss: 0.0694 - accuracy: 0.9732 - val_loss: 0.0582 - val_accuracy: 0.9805 - lr: 0.0010
Epoch 5/100
854/854 [==============================] - 16s 18ms/step - loss: 0.0545 - accuracy: 0.9789 - val_loss: 0.0598 - val_accuracy: 0.9793 - lr: 0.0010
Epoch 6/100
854/854 [==============================] - 15s 18ms/step - loss: 0.0504 - accuracy: 0.9816 - val_loss: 0.0523 - val_accuracy: 0.9807 - lr: 0.0010
Epoch 7/100
854/854 [==============================]

# C MERT v1 330M

In [9]:
train_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/MERT-330/Chinese_train_MERT-v1-330M/Chinese_train_MERT-v1-330M.csv'
dev_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/MERT-330/Chinese_dev_MERT-v1-330M/Chinese_dev_MERT-v1-330M.csv'
test_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/MERT-330/Chinese_test_MERT-v1-330M/Chinese_test_MERT-v1-330M.csv'

# Call the function to train and evaluate the CNN model
cnn_model = train_and_evaluate_cnn(train_file, dev_file, test_file, epochs=100, batch_size=32)


Epoch 1/100
854/854 [==============================] - 22s 24ms/step - loss: 0.2549 - accuracy: 0.8900 - val_loss: 0.1041 - val_accuracy: 0.9625 - lr: 0.0010
Epoch 2/100
854/854 [==============================] - 19s 23ms/step - loss: 0.0961 - accuracy: 0.9637 - val_loss: 0.0617 - val_accuracy: 0.9808 - lr: 0.0010
Epoch 3/100
854/854 [==============================] - 21s 24ms/step - loss: 0.0742 - accuracy: 0.9723 - val_loss: 0.0640 - val_accuracy: 0.9784 - lr: 0.0010
Epoch 4/100
854/854 [==============================] - 19s 23ms/step - loss: 0.0683 - accuracy: 0.9740 - val_loss: 0.0488 - val_accuracy: 0.9841 - lr: 0.0010
Epoch 5/100
854/854 [==============================] - 20s 24ms/step - loss: 0.0586 - accuracy: 0.9782 - val_loss: 0.1163 - val_accuracy: 0.9514 - lr: 0.0010
Epoch 6/100
854/854 [==============================] - 20s 23ms/step - loss: 0.0576 - accuracy: 0.9790 - val_loss: 0.0656 - val_accuracy: 0.9765 - lr: 0.0010
Epoch 7/100
854/854 [==============================]

# C wav2vec2 

In [10]:
train_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/Wav2Vec2/Chinese_train_Wav2Vec2.csv'
dev_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/Wav2Vec2/Chinese_dev_Wav2Vec2.csv'
test_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/Wav2Vec2/Chinese_test_Wav2Vec2.csv'

# Call the function to train and evaluate the CNN model
cnn_model = train_and_evaluate_cnn(train_file, dev_file, test_file, epochs=100, batch_size=32)


Epoch 1/100
854/854 [==============================] - 15s 16ms/step - loss: 0.0288 - accuracy: 0.9906 - val_loss: 0.0028 - val_accuracy: 0.9997 - lr: 0.0010
Epoch 2/100
854/854 [==============================] - 13s 16ms/step - loss: 0.0112 - accuracy: 0.9964 - val_loss: 0.0056 - val_accuracy: 0.9984 - lr: 0.0010
Epoch 3/100
854/854 [==============================] - 13s 16ms/step - loss: 0.0121 - accuracy: 0.9960 - val_loss: 0.0031 - val_accuracy: 0.9996 - lr: 0.0010
Epoch 4/100
854/854 [==============================] - 13s 16ms/step - loss: 0.0078 - accuracy: 0.9975 - val_loss: 0.0388 - val_accuracy: 0.9845 - lr: 0.0010
Epoch 5/100
854/854 [==============================] - 14s 16ms/step - loss: 0.0069 - accuracy: 0.9978 - val_loss: 0.0094 - val_accuracy: 0.9975 - lr: 5.0000e-04
Epoch 6/100
854/854 [==============================] - 12s 14ms/step - loss: 0.0047 - accuracy: 0.9984 - val_loss: 0.0151 - val_accuracy: 0.9954 - lr: 5.0000e-04
Epoch 7/100
854/854 [=======================

# E wav2vec2 

In [11]:
train_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/Wav2Vec2/English_train_Wav2Vec2.csv'
dev_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/Wav2Vec2/English_dev_Wav2Vec2.csv'
test_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/Wav2Vec2/English_test_Wav2Vec2.csv'

# Call the function to train and evaluate the CNN model
cnn_model = train_and_evaluate_cnn(train_file, dev_file, test_file, epochs=100, batch_size=32)


Epoch 1/100
854/854 [==============================] - 11s 12ms/step - loss: 0.0277 - accuracy: 0.9906 - val_loss: 0.0017 - val_accuracy: 0.9999 - lr: 0.0010
Epoch 2/100
854/854 [==============================] - 10s 12ms/step - loss: 0.0154 - accuracy: 0.9952 - val_loss: 0.0222 - val_accuracy: 0.9927 - lr: 0.0010
Epoch 3/100
854/854 [==============================] - 10s 12ms/step - loss: 0.0095 - accuracy: 0.9967 - val_loss: 0.0016 - val_accuracy: 0.9998 - lr: 0.0010
Epoch 4/100
854/854 [==============================] - 10s 12ms/step - loss: 0.0092 - accuracy: 0.9971 - val_loss: 0.0019 - val_accuracy: 0.9998 - lr: 0.0010
Epoch 5/100
854/854 [==============================] - 10s 12ms/step - loss: 0.0066 - accuracy: 0.9978 - val_loss: 2.7805e-04 - val_accuracy: 1.0000 - lr: 5.0000e-04
Epoch 6/100
854/854 [==============================] - 10s 12ms/step - loss: 0.0069 - accuracy: 0.9979 - val_loss: 5.6054e-04 - val_accuracy: 0.9999 - lr: 5.0000e-04
Epoch 7/100
854/854 [===============

# C unispeech

In [12]:
train_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/UNISPEECH/Chinese_train_UniSpeechSAT/Chinese_train_UniSpeechSAT.csv'
dev_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/UNISPEECH/Chinese_dev_UniSpeechSAT/Chinese_dev_UniSpeechSAT.csv'
test_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/UNISPEECH/Chinese_test_UniSpeechSAT/Chinese_test_UniSpeechSAT.csv'


cnn_model = train_and_evaluate_cnn(train_file, dev_file, test_file, epochs=100, batch_size=32)


Epoch 1/100
854/854 [==============================] - 12s 12ms/step - loss: 0.0502 - accuracy: 0.9816 - val_loss: 0.1369 - val_accuracy: 0.9519 - lr: 0.0010
Epoch 2/100
854/854 [==============================] - 10s 12ms/step - loss: 0.0264 - accuracy: 0.9913 - val_loss: 0.0302 - val_accuracy: 0.9907 - lr: 0.0010
Epoch 3/100
854/854 [==============================] - 10s 12ms/step - loss: 0.0246 - accuracy: 0.9915 - val_loss: 0.0560 - val_accuracy: 0.9816 - lr: 0.0010
Epoch 4/100
854/854 [==============================] - 10s 12ms/step - loss: 0.0254 - accuracy: 0.9915 - val_loss: 0.0171 - val_accuracy: 0.9955 - lr: 0.0010
Epoch 5/100
854/854 [==============================] - 10s 12ms/step - loss: 0.0240 - accuracy: 0.9919 - val_loss: 0.0260 - val_accuracy: 0.9929 - lr: 0.0010
Epoch 6/100
854/854 [==============================] - 10s 12ms/step - loss: 0.0234 - accuracy: 0.9920 - val_loss: 0.0180 - val_accuracy: 0.9946 - lr: 0.0010
Epoch 7/100
854/854 [==============================]

# E Unispeech

In [13]:
train_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/UNISPEECH/English_train_UniSpeechSAT/English_train_UniSpeechSAT.csv'
dev_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/UNISPEECH/English_dev_UniSpeechSAT/English_dev_UniSpeechSAT.csv'
test_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/UNISPEECH/English_test_UniSpeechSAT/English_test_UniSpeechSAT.csv'


cnn_model = train_and_evaluate_cnn(train_file, dev_file, test_file, epochs=100, batch_size=32)


Epoch 1/100
854/854 [==============================] - 13s 13ms/step - loss: 0.0629 - accuracy: 0.9779 - val_loss: 0.0074 - val_accuracy: 0.9980 - lr: 0.0010
Epoch 2/100
854/854 [==============================] - 10s 12ms/step - loss: 0.0330 - accuracy: 0.9884 - val_loss: 0.0055 - val_accuracy: 0.9982 - lr: 0.0010
Epoch 3/100
854/854 [==============================] - 10s 12ms/step - loss: 0.0285 - accuracy: 0.9896 - val_loss: 0.0094 - val_accuracy: 0.9970 - lr: 0.0010
Epoch 4/100
854/854 [==============================] - 10s 12ms/step - loss: 0.0257 - accuracy: 0.9913 - val_loss: 0.0239 - val_accuracy: 0.9913 - lr: 0.0010
Epoch 5/100
854/854 [==============================] - 10s 12ms/step - loss: 0.0248 - accuracy: 0.9912 - val_loss: 0.0027 - val_accuracy: 0.9992 - lr: 0.0010
Epoch 6/100
854/854 [==============================] - 10s 12ms/step - loss: 0.0237 - accuracy: 0.9914 - val_loss: 0.0109 - val_accuracy: 0.9966 - lr: 0.0010
Epoch 7/100
854/854 [==============================]

# 

# E MERT-v1-95M

In [19]:
train_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/MERT-v1-95M/English_train_MERT-v1-95M/English_train_MERT-v1-95M.csv'
dev_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/MERT-v1-95M/English_dev_MERT-v1-95M/English_dev_MERT-v1-95M.csv'
test_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/MERT-v1-95M/English_test_MERT-v1-95M/English_test_MERT-v1-95M.csv'


cnn_model = train_and_evaluate_cnn(train_file, dev_file, test_file, epochs=50, batch_size=32)


Epoch 1/50
854/854 [==============================] - 13s 14ms/step - loss: 0.3950 - accuracy: 0.8220 - val_loss: 0.1352 - val_accuracy: 0.9508 - lr: 0.0010
Epoch 2/50
854/854 [==============================] - 11s 13ms/step - loss: 0.1387 - accuracy: 0.9471 - val_loss: 0.0707 - val_accuracy: 0.9765 - lr: 0.0010
Epoch 3/50
854/854 [==============================] - 11s 13ms/step - loss: 0.1059 - accuracy: 0.9592 - val_loss: 0.0691 - val_accuracy: 0.9742 - lr: 0.0010
Epoch 4/50
854/854 [==============================] - 11s 13ms/step - loss: 0.0869 - accuracy: 0.9674 - val_loss: 0.0769 - val_accuracy: 0.9715 - lr: 0.0010
Epoch 5/50
854/854 [==============================] - 11s 13ms/step - loss: 0.0693 - accuracy: 0.9742 - val_loss: 0.0579 - val_accuracy: 0.9785 - lr: 0.0010
Epoch 6/50
854/854 [==============================] - 11s 13ms/step - loss: 0.0693 - accuracy: 0.9740 - val_loss: 0.0477 - val_accuracy: 0.9830 - lr: 0.0010
Epoch 7/50
854/854 [==============================] - 11s 

# C MERT-v1-95M

In [20]:
train_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/MERT-v1-95M/Chinese_train_MERT-v1-95M/Chinese_train_MERT-v1-95M.csv'
dev_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/MERT-v1-95M/Chinese_dev_MERT-v1-95M/Chinese_dev_MERT-v1-95M.csv'
test_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/MERT-v1-95M/Chinese_test_MERT-v1-95M/Chinese_test_MERT-v1-95M.csv'


cnn_model = train_and_evaluate_cnn(train_file, dev_file, test_file, epochs=50, batch_size=32)


Epoch 1/50
854/854 [==============================] - 14s 14ms/step - loss: 0.3507 - accuracy: 0.8400 - val_loss: 0.1408 - val_accuracy: 0.9503 - lr: 0.0010
Epoch 2/50
854/854 [==============================] - 12s 14ms/step - loss: 0.1095 - accuracy: 0.9585 - val_loss: 0.0869 - val_accuracy: 0.9664 - lr: 0.0010
Epoch 3/50
854/854 [==============================] - 11s 13ms/step - loss: 0.0842 - accuracy: 0.9681 - val_loss: 0.1053 - val_accuracy: 0.9598 - lr: 0.0010
Epoch 4/50
854/854 [==============================] - 11s 13ms/step - loss: 0.0706 - accuracy: 0.9732 - val_loss: 0.0975 - val_accuracy: 0.9677 - lr: 0.0010
Epoch 5/50
854/854 [==============================] - 11s 13ms/step - loss: 0.0624 - accuracy: 0.9765 - val_loss: 0.0644 - val_accuracy: 0.9774 - lr: 0.0010
Epoch 6/50
854/854 [==============================] - 11s 13ms/step - loss: 0.0701 - accuracy: 0.9733 - val_loss: 0.0961 - val_accuracy: 0.9638 - lr: 0.0010
Epoch 7/50
854/854 [==============================] - 11s 

# C MERT-v0-public

In [23]:
train_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/MERT-v0-public/Chinese_train_MERT-v0-public/Chinese_train_MERT-v0-public.csv'
dev_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/MERT-v0-public/Chinese_dev_MERT-v0-public/Chinese_dev_MERT-v0-public.csv'
test_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/MERT-v0-public/Chinese_test_MERT-v0-public/Chinese_test_MERT-v0-public.csv'


cnn_model = train_and_evaluate_cnn(train_file, dev_file, test_file, epochs=50, batch_size=32)


Epoch 1/50
854/854 [==============================] - 13s 14ms/step - loss: 0.3619 - accuracy: 0.8433 - val_loss: 0.1365 - val_accuracy: 0.9488 - lr: 0.0010
Epoch 2/50
854/854 [==============================] - 11s 13ms/step - loss: 0.0953 - accuracy: 0.9637 - val_loss: 0.1148 - val_accuracy: 0.9571 - lr: 0.0010
Epoch 3/50
854/854 [==============================] - 11s 13ms/step - loss: 0.0796 - accuracy: 0.9706 - val_loss: 0.1502 - val_accuracy: 0.9503 - lr: 0.0010
Epoch 4/50
854/854 [==============================] - 11s 13ms/step - loss: 0.0677 - accuracy: 0.9741 - val_loss: 0.0822 - val_accuracy: 0.9696 - lr: 0.0010
Epoch 5/50
854/854 [==============================] - 11s 13ms/step - loss: 0.0606 - accuracy: 0.9775 - val_loss: 0.0797 - val_accuracy: 0.9693 - lr: 0.0010
Epoch 6/50
854/854 [==============================] - 11s 13ms/step - loss: 0.0485 - accuracy: 0.9829 - val_loss: 0.0808 - val_accuracy: 0.9700 - lr: 0.0010
Epoch 7/50
854/854 [==============================] - 11s 

# E MERT-v0-public

In [24]:
train_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/MERT-v0-public/English_train_MERT-v0-public/English_train_MERT-v0-public.csv'
dev_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/MERT-v0-public/English_dev_MERT-v0-public/English_dev_MERT-v0-public.csv'
test_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/MERT-v0-public/English_test_MERT-v0-public/English_test_MERT-v0-public.csv'


cnn_model = train_and_evaluate_cnn(train_file, dev_file, test_file, epochs=50, batch_size=32)


Epoch 1/50
854/854 [==============================] - 13s 13ms/step - loss: 0.3666 - accuracy: 0.8352 - val_loss: 0.1186 - val_accuracy: 0.9524 - lr: 0.0010
Epoch 2/50
854/854 [==============================] - 11s 13ms/step - loss: 0.0996 - accuracy: 0.9604 - val_loss: 0.0516 - val_accuracy: 0.9818 - lr: 0.0010
Epoch 3/50
854/854 [==============================] - 11s 13ms/step - loss: 0.0745 - accuracy: 0.9727 - val_loss: 0.0825 - val_accuracy: 0.9671 - lr: 0.0010
Epoch 4/50
854/854 [==============================] - 11s 13ms/step - loss: 0.0659 - accuracy: 0.9754 - val_loss: 0.0449 - val_accuracy: 0.9836 - lr: 0.0010
Epoch 5/50
854/854 [==============================] - 11s 13ms/step - loss: 0.0605 - accuracy: 0.9773 - val_loss: 0.0350 - val_accuracy: 0.9884 - lr: 0.0010
Epoch 6/50
854/854 [==============================] - 11s 13ms/step - loss: 0.0546 - accuracy: 0.9799 - val_loss: 0.0591 - val_accuracy: 0.9788 - lr: 0.0010
Epoch 7/50
854/854 [==============================] - 11s 

# 

# E mfcc

In [2]:
train_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/MFCC/English_train_MFCC.csv'
dev_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/MFCC/English_dev_MFCC.csv'
test_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/MFCC/English_test_MFCC.csv'


cnn_model = train_and_evaluate_cnn(train_file, dev_file, test_file, epochs=50, batch_size=32)


2024-10-31 04:29:31.688156: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-31 04:29:31.884761: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-31 04:29:31.884846: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-31 04:29:31.888220: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-31 04:29:31.888265: I external/local_xla/xla/stream_executor

Epoch 1/50


2024-10-31 04:29:33.217777: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906
2024-10-31 04:29:33.478766: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-10-31 04:29:33.908867: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-10-31 04:29:34.794646: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f2dbb08acb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-10-31 04:29:34.794691: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 6GB Laptop GPU, Compute Capability 8.6
2024-10-31 04:29:34.811982: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1730329174.907732   56294 d

854/854 [==============================] - 12s 10ms/step - loss: 0.1102 - accuracy: 0.9595 - val_loss: 0.9179 - val_accuracy: 0.7513 - lr: 0.0010
Epoch 2/50
854/854 [==============================] - 7s 9ms/step - loss: 0.0564 - accuracy: 0.9794 - val_loss: 0.4126 - val_accuracy: 0.8568 - lr: 0.0010
Epoch 3/50
854/854 [==============================] - 8s 9ms/step - loss: 0.0488 - accuracy: 0.9823 - val_loss: 0.4563 - val_accuracy: 0.8630 - lr: 0.0010
Epoch 4/50
854/854 [==============================] - 7s 8ms/step - loss: 0.0397 - accuracy: 0.9852 - val_loss: 0.5590 - val_accuracy: 0.8352 - lr: 0.0010
Epoch 5/50
854/854 [==============================] - 7s 8ms/step - loss: 0.0331 - accuracy: 0.9880 - val_loss: 0.4510 - val_accuracy: 0.8648 - lr: 0.0010
Epoch 6/50
854/854 [==============================] - 8s 9ms/step - loss: 0.0304 - accuracy: 0.9896 - val_loss: 0.5222 - val_accuracy: 0.8508 - lr: 5.0000e-04
Epoch 7/50
854/854 [==============================] - 8s 10ms/step - loss: 

# C MFCC

In [6]:
train_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/MFCC/Chinese_train_MFCC.csv'
dev_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/MFCC/Chinese_dev_MFCC.csv'
test_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/MFCC/Chinese_test_MFCC.csv'


cnn_model = train_and_evaluate_cnn(train_file, dev_file, test_file, epochs=50, batch_size=32)


Epoch 1/50
854/854 [==============================] - 21s 18ms/step - loss: 0.1052 - accuracy: 0.9613 - val_loss: 0.1206 - val_accuracy: 0.9565 - lr: 0.0010
Epoch 2/50
854/854 [==============================] - 15s 17ms/step - loss: 0.0408 - accuracy: 0.9857 - val_loss: 0.0829 - val_accuracy: 0.9701 - lr: 0.0010
Epoch 3/50
854/854 [==============================] - 15s 17ms/step - loss: 0.0330 - accuracy: 0.9884 - val_loss: 0.1093 - val_accuracy: 0.9640 - lr: 0.0010
Epoch 4/50
854/854 [==============================] - 15s 17ms/step - loss: 0.0267 - accuracy: 0.9902 - val_loss: 0.0827 - val_accuracy: 0.9704 - lr: 0.0010
Epoch 5/50
854/854 [==============================] - 15s 18ms/step - loss: 0.0223 - accuracy: 0.9927 - val_loss: 0.0458 - val_accuracy: 0.9830 - lr: 0.0010
Epoch 6/50
854/854 [==============================] - 14s 17ms/step - loss: 0.0253 - accuracy: 0.9911 - val_loss: 0.1014 - val_accuracy: 0.9633 - lr: 0.0010
Epoch 7/50
854/854 [==============================] - 14s 

# E music2vec1

In [1]:
train_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/music2vec1/English_train_music2vec.csv'
dev_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/music2vec1/English_dev_music2vec.csv'
test_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/music2vec1/English_test_music2vec.csv'


cnn_model = train_and_evaluate_cnn(train_file, dev_file, test_file, epochs=50, batch_size=32)


2024-11-01 01:27:17.371451: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-01 01:27:17.772246: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-01 01:27:17.772305: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-01 01:27:17.838930: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-01 01:27:17.979197: I tensorflow/core/platform/cpu_feature_guar

Epoch 1/50


2024-11-01 01:27:28.775254: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906
2024-11-01 01:27:29.060297: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-11-01 01:27:29.522890: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-11-01 01:27:30.588781: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f89ff16bc90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-11-01 01:27:30.588819: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 6GB Laptop GPU, Compute Capability 8.6
2024-11-01 01:27:30.605833: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1730404650.690986  465105 d

854/854 [==============================] - 17s 16ms/step - loss: 0.0152 - accuracy: 0.9950 - val_loss: 41.5195 - val_accuracy: 0.6154 - lr: 0.0010
Epoch 2/50
854/854 [==============================] - 12s 14ms/step - loss: 0.0070 - accuracy: 0.9981 - val_loss: 35.0328 - val_accuracy: 0.4643 - lr: 0.0010
Epoch 3/50
854/854 [==============================] - 12s 15ms/step - loss: 0.0041 - accuracy: 0.9985 - val_loss: 550.9406 - val_accuracy: 0.3846 - lr: 0.0010
Epoch 4/50
854/854 [==============================] - 13s 15ms/step - loss: 0.0047 - accuracy: 0.9988 - val_loss: 1294.5963 - val_accuracy: 0.3846 - lr: 0.0010
Epoch 5/50
854/854 [==============================] - 13s 15ms/step - loss: 0.0049 - accuracy: 0.9984 - val_loss: 2141.8660 - val_accuracy: 0.3846 - lr: 0.0010
Epoch 6/50
854/854 [==============================] - 13s 15ms/step - loss: 0.0020 - accuracy: 0.9994 - val_loss: 2496.4321 - val_accuracy: 0.3846 - lr: 5.0000e-04
Epoch 7/50
854/854 [==============================] 

/home/girish/anaconda3/envs/tf_gpu/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/girish/anaconda3/envs/tf_gpu/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/girish/anaconda3/envs/tf_gpu/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitaliz

# C music2vec1

In [6]:
train_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/music2vec1/Chinese_train_music2vec.csv'
dev_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/music2vec1/Chinese_dev_music2vec.csv'
test_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/music2vec1/Chinese_test_music2vec.csv'


cnn_model = train_and_evaluate_cnn(train_file, dev_file, test_file, epochs=50, batch_size=32)


Epoch 1/50
854/854 [==============================] - 14s 15ms/step - loss: 0.0298 - accuracy: 0.9897 - val_loss: 253.3068 - val_accuracy: 0.6154 - lr: 0.0010
Epoch 2/50
854/854 [==============================] - 12s 14ms/step - loss: 0.0108 - accuracy: 0.9967 - val_loss: 556.3622 - val_accuracy: 0.3846 - lr: 0.0010
Epoch 3/50
854/854 [==============================] - 12s 14ms/step - loss: 0.0111 - accuracy: 0.9966 - val_loss: 2122.4456 - val_accuracy: 0.3846 - lr: 0.0010
Epoch 4/50
854/854 [==============================] - 12s 14ms/step - loss: 0.0093 - accuracy: 0.9971 - val_loss: 1549.2235 - val_accuracy: 0.3846 - lr: 0.0010
Epoch 5/50
854/854 [==============================] - 12s 14ms/step - loss: 0.0038 - accuracy: 0.9988 - val_loss: 2351.3501 - val_accuracy: 0.3846 - lr: 5.0000e-04
Epoch 6/50
854/854 [==============================] - 12s 14ms/step - loss: 0.0037 - accuracy: 0.9989 - val_loss: 1830.5482 - val_accuracy: 0.3846 - lr: 5.0000e-04
Epoch 7/50
854/854 [==============

/home/girish/anaconda3/envs/tf_gpu/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/girish/anaconda3/envs/tf_gpu/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/girish/anaconda3/envs/tf_gpu/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitaliz

# C MERT v0

In [3]:

train_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/MERT-v0/Chinese_train_MERT-v0/Chinese_train_MERT-v0.csv'
dev_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/MERT-v0/Chinese_dev_MERT-v0/Chinese_dev_MERT-v0.csv'
test_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/MERT-v0/Chinese_test_MERT-v0/Chinese_test_MERT-v0.csv'


cnn_model = train_and_evaluate_cnn(train_file, dev_file, test_file, epochs=50, batch_size=32)


Epoch 1/50
854/854 [==============================] - 14s 15ms/step - loss: 0.4519 - accuracy: 0.7957 - val_loss: 0.2195 - val_accuracy: 0.9154 - lr: 0.0010
Epoch 2/50
854/854 [==============================] - 12s 14ms/step - loss: 0.1524 - accuracy: 0.9382 - val_loss: 0.1546 - val_accuracy: 0.9402 - lr: 0.0010
Epoch 3/50
854/854 [==============================] - 12s 14ms/step - loss: 0.1185 - accuracy: 0.9542 - val_loss: 0.1569 - val_accuracy: 0.9367 - lr: 0.0010
Epoch 4/50
854/854 [==============================] - 13s 15ms/step - loss: 0.1026 - accuracy: 0.9603 - val_loss: 0.2009 - val_accuracy: 0.9165 - lr: 0.0010
Epoch 5/50
854/854 [==============================] - 13s 15ms/step - loss: 0.0988 - accuracy: 0.9630 - val_loss: 0.1049 - val_accuracy: 0.9602 - lr: 0.0010
Epoch 6/50
854/854 [==============================] - 13s 15ms/step - loss: 0.0774 - accuracy: 0.9708 - val_loss: 0.1356 - val_accuracy: 0.9496 - lr: 0.0010
Epoch 7/50
854/854 [==============================] - 13s 

# E MERT v0

In [4]:
train_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/MERT-v0/English_train_MERT-v0/English_train_MERT-v0.csv'
dev_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/MERT-v0/English_dev_MERT-v0/English_dev_MERT-v0.csv'
test_file = '/home/girish/Girish/RESEARCH/EMO-FAKE/FEATURES/MERT-v0/English_test_MERT-v0/English_test_MERT-v0.csv'


cnn_model = train_and_evaluate_cnn(train_file, dev_file, test_file, epochs=50, batch_size=32)


Epoch 1/50
854/854 [==============================] - 13s 14ms/step - loss: 0.4460 - accuracy: 0.8032 - val_loss: 0.1487 - val_accuracy: 0.9418 - lr: 0.0010
Epoch 2/50
854/854 [==============================] - 11s 13ms/step - loss: 0.1500 - accuracy: 0.9410 - val_loss: 0.0988 - val_accuracy: 0.9613 - lr: 0.0010
Epoch 3/50
854/854 [==============================] - 12s 14ms/step - loss: 0.1266 - accuracy: 0.9511 - val_loss: 0.1356 - val_accuracy: 0.9451 - lr: 0.0010
Epoch 4/50
854/854 [==============================] - 11s 13ms/step - loss: 0.1136 - accuracy: 0.9562 - val_loss: 0.0903 - val_accuracy: 0.9670 - lr: 0.0010
Epoch 5/50
854/854 [==============================] - 12s 14ms/step - loss: 0.0996 - accuracy: 0.9621 - val_loss: 0.0880 - val_accuracy: 0.9652 - lr: 0.0010
Epoch 6/50
854/854 [==============================] - 12s 14ms/step - loss: 0.0903 - accuracy: 0.9658 - val_loss: 0.0923 - val_accuracy: 0.9636 - lr: 0.0010
Epoch 7/50
854/854 [==============================] - 12s 